<a href="https://colab.research.google.com/github/flora0110/sentence_prediction/blob/main/bge_m3_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
!pip install udicOpenData

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for udicOpenData: filename=udicOpenData-2.4-py3-none-any.whl size=18094423 sha256=9da411b035cd58c1412ee6a0365aa454bcafc644c750401141e4a88524f0b3ef
  Stored in directory: /root/.cache/pip/wheels/86/4f/46/2a28bc28485ba1f9d67dead62ac6aba14089a6cf1d5fbad365
Successfully built udicOpenData


In [3]:
!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 5.8 MB/s eta 0:00:00


In [17]:
!pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 16.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_c

In [25]:
import numpy as np
import json
from opencc import OpenCC
from sklearn.neighbors import NearestNeighbors
import pickle

In [5]:
from udicOpenData.stopwords import *

In [20]:
from FlagEmbedding import BGEM3FlagModel

# Data import

In [7]:
folder_path = "/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/"

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
with open(folder_path+'train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(folder_path+'test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [10]:
def extract_key_contents(data):
    facts = []
    sentences = []
    for item in data:
        facts.append(item.get('fact'))
        sentences.append(item.get('sentance'))
    return facts, sentences

In [11]:
train_facts, train_sentences = extract_key_contents(train_data)
test_facts, test_sentences = extract_key_contents(test_data)

In [12]:
def read_string_array_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            string_array = file.readlines()
            # 去除每個元素末尾的換行符
            string_array = [string.strip() for string in string_array]
        print(f"String array read from {file_path} successfully.")
        return string_array
    except Exception as e:
        print(f"Error occurred while reading string array: {e}")
        return None

In [13]:
train_summaries = read_string_array_from_file(folder_path + 'train_summaries.txt')
test_summaries = read_string_array_from_file(folder_path + 'test_summaries.txt')

String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/train_summaries.txt successfully.
String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/test_summaries.txt successfully.


# Preprocessing

## 斷詞

In [14]:
cc = OpenCC('t2s')

In [15]:
# 分詞加空白
def split_passage(passage):
    word_and_tag = list(rmsw(passage, flag=True))
    _temp = list(filter(lambda x: x[1]!='nr' and x[1]!='ns', word_and_tag))
    # print(_temp)
    text = " ".join(list(map(lambda x:x[0], _temp)))
    return text

In [ ]:
# tokenized_train_summaries = []

# for data in train_summaries:
#     tokenized_train_summaries.append(split_passage(cc.convert(data)))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.977 seconds.
DEBUG:jieba:Loading model cost 0.977 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# tokenized_test_summaries = []

# for data in test_summaries:
#     tokenized_test_summaries.append(split_passage(cc.convert(data)))

In [ ]:
def save_string_array_to_file(string_array, file_path):
    try:
        with open(file_path, 'w') as file:
            for string in string_array:
                file.write(string + '\n')
        print(f"String array saved to {file_path} successfully.")
    except Exception as e:
        print(f"Error occurred while saving string array: {e}")

In [ ]:
# save_string_array_to_file(tokenized_train_summaries, '/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt')
# save_string_array_to_file(tokenized_test_summaries, '/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt')

String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt successfully.
String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt successfully.


In [16]:
tokenized_train_summaries = read_string_array_from_file('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt')
tokenized_test_summaries = read_string_array_from_file('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt')

String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt successfully.
String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt successfully.


## Data embedding

In [21]:
model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

colbert_linear.pt:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

In [22]:
sword_1 = ["老师", "商品"]
sword_2 = ["教师", "货物"]

## dense

### 測試在中文上的效果

In [23]:
embeddings_1 = model.encode(sword_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sword_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.8509769  0.5041799 ]
 [0.464958   0.74887514]]


### train test data embedding

In [ ]:
# train_embeddings_all = model.encode(tokenized_train_summaries, max_length=1500)

In [ ]:
# test_embeddings_all = model.encode(tokenized_test_summaries, max_length=1500)

In [ ]:
# with open(folder_path + 'train_embeddings_all.pkl', 'wb') as f:
#     pickle.dump(train_embeddings_all, f)

In [ ]:
# with open(folder_path + 'test_embeddings_all.pkl', 'wb') as f:
#     pickle.dump(test_embeddings_all, f)

In [26]:
with open(folder_path + 'train_embeddings_all.pkl', 'rb') as f:
    train_embeddings_all = pickle.load(f)

In [28]:
train_embeddings_all['dense_vecs']

array([[-0.03034765,  0.02622485, -0.01174086, ..., -0.01362408,
        -0.04923247, -0.0444256 ],
       [-0.02140026,  0.0173902 , -0.06489349, ..., -0.00908999,
        -0.02669849,  0.02307938],
       [-0.01579434,  0.04073653, -0.02186119, ..., -0.02366436,
        -0.0657978 , -0.05442155],
       ...,
       [-0.05367578,  0.01930024, -0.00059708, ...,  0.015949  ,
        -0.05102739,  0.01105056],
       [-0.01177966,  0.02777622, -0.02606803, ..., -0.01549191,
        -0.04460454, -0.01327729],
       [-0.0269404 ,  0.02274233, -0.03002858, ...,  0.02050726,
        -0.04272795,  0.0081914 ]], dtype=float32)

In [ ]:
# train_embeddings = model.encode(tokenized_train_summaries, max_length=1500)['dense_vecs']

Inference Embeddings:  81%|████████  | 34/42 [1:20:17<19:33, 146.66s/it]

In [ ]:
# test_embeddings = model.encode(tokenized_test_summaries, max_length=1500)['dense_vecs']

In [ ]:
# np.savetxt(folder_path + 'train_embeddings.txt', train_embeddings)
# np.savetxt(folder_path + 'test_embeddings.txt', test_embeddings)

In [ ]:
train_embeddings = np.loadtxt(folder_path + 'train_embeddings.txt')
test_embeddings = np.loadtxt(folder_path + 'test_embeddings.txt')

In [ ]:
type(train_embeddings)

numpy.ndarray

In [ ]:
train_embeddings

array([[-0.03034765,  0.02622485, -0.01174086, ..., -0.01362408,
        -0.04923247, -0.0444256 ],
       [-0.02140026,  0.0173902 , -0.06489349, ..., -0.00908999,
        -0.02669849,  0.02307938],
       [-0.01579434,  0.04073653, -0.02186119, ..., -0.02366436,
        -0.0657978 , -0.05442155],
       ...,
       [-0.05367578,  0.01930024, -0.00059708, ...,  0.015949  ,
        -0.05102739,  0.01105056],
       [-0.01177966,  0.02777622, -0.02606803, ..., -0.01549191,
        -0.04460454, -0.01327729],
       [-0.0269404 ,  0.02274233, -0.03002858, ...,  0.02050726,
        -0.04272795,  0.0081914 ]])

In [ ]:
print(test_embeddings.shape)
print(test_embeddings[0].shape)
print(test_embeddings[0].reshape(-1, 1).shape)

(100, 1024)
(1024,)
(1024, 1)


### KNN

In [ ]:
k = 5
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine')

In [ ]:
# 將模型保存到文件
with open(folder_path + 'knn_model_SBERT.pkl', 'wb') as f:
    pickle.dump(knn_model_SBERT, f)

In [ ]:
# 從文件加載模型
with open(folder_path + 'knn_model_SBERT.pkl', 'rb') as f:
    knn_model_SBERT_loaded = pickle.load(f)

### 以第一篇test fact進行測試

In [ ]:
distances, indices = knn_model_SBERT_loaded.kneighbors(test_embeddings[0].reshape(1, -1))

In [ ]:
indices

array([[415,  47, 114, 250, 137]])

In [ ]:
distances

array([[0.22361922, 0.26161647, 0.27732217, 0.28204942, 0.2864439 ]],
      dtype=float32)

In [ ]:
test_facts[0]

'○、緣徐鴻光（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決處有期徒刑3月、緩刑3年確定）因需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，知悉車牌號碼00-000號營業小客車係徐鴻光向志朗交通有限公司（下稱志朗公司）租賃之車輛，竟由穆立明先於不詳時間、不詳地點，偽刻「志朗交通有限公司」及負責人「陳健志」印章（正確姓名為陳建志，誤刻為「陳健志」）各1枚，並偽造「陳健志」代表志朗公司與徐鴻光簽署之「臺北市計程車客運駕駛人自備車輛參與經營契約書」（下稱本案參與經營契約書），虛偽記載徐鴻光提供上述營業小客車靠行志朗公司營業等不實內容，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名，2人復於同日協同不知情之徐鴻光配偶徐樂仁（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決無罪確定），前往臺北市○○區○○街00巷0號2樓王傑生所經營之宏帝當舖（現更名為渣打當舖），由徐鴻光、穆立明交付偽造本案參與經營契約書向宏帝當鋪借款新臺幣（下同）10萬元以為行使，並約定由徐樂仁擔任連帶保證人，王傑生誤信上開車輛為徐鴻光所有而陷入錯誤，於扣除利息後交付9萬1千元借款予徐鴻光夫婦，足以生損害於志朗公司、陳建志、王傑生。嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。理由○、證據能力部分：被告以外之人於審判外之言詞或書面陳述，除法律有規定者外，不得作為證據；又被告以外之人於審判外之陳述，雖不符合同法第159條之1至第159條之4之規定，但經當事人於審判程序同意作為證據，法院審酌該言詞陳述或書面陳述作成時之情況，認為適當者，亦得為證據。刑事訴訟法第159條第1項及第159條之5第1項分別定有明文。經查，本判決所引用之各該被告以外之人於審判外之陳述，雖屬傳聞證據，然當事人於本院審判期日中均表示同意作為證據方法而不予爭執，且迄至言詞辯論終結前亦未再聲明異議（見本院卷第68頁至第68頁背面），本院審酌上開證據資料製作時之情況，尚無違法不當及證明力明顯過低之瑕疵，亦認為以之作為證據應屬適當

In [ ]:
test_summaries[0]

'因徐鴻光需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，偽造「陳健志」代表志朗公司與徐鴻光簽署的「臺北市計程車客運駕駛人自備車輛參與經營契約書」，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名。嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。'

In [ ]:
# 打印最相近的五篇文章的索引及距離
for i, index in enumerate(indices[0]):
    print("Article Index:", index, "Distance:", distances[0][i])
    print("Article Text:", train_facts[index])

Article Index: 415 Distance: 0.22361922
Article Text: 一、緣陳榮煉於民國106年7月間，借用李美娟之名義，以分期付款方式購入車牌號碼000-0000號自小客車後，僅繳納16期分期價金後，即因經濟困難而無力繳清餘款，而導致李美娟屢遭銀行催繳遲延價金（所涉詐欺罪嫌另由檢察官為不起訴處分），李美娟遂要求陳榮煉儘速將車輛讓渡，且不得再以其名義及該車輛借款。嗣陳榮煉復欲向第三人借款新臺幣(下同)14萬元，竟未得李美娟同意，基於偽造文書之犯意，於106年12月20日某時，在不詳地點偽造「李美娟」之署名1枚填載於「讓渡證書」之「立讓渡書人」欄位上，連同行車執照及本票2紙交由該第三人後行使之以作為其借款擔保，足以生損害於李美娟之權益。二、案經李美娟訴請臺灣嘉義地方檢察署檢察官偵查起訴。理由壹、程序部分：一、檢察官及被告陳榮煉對於下述本院採為認定犯罪事實依據之各項傳聞證據之證據能力，均表示同意有證據能力（見本院卷第31頁、第47頁、第48頁），另於言詞辯論終結前，除經同意有證據能力之證據外，對於卷附各傳聞證據之證據能力，均未聲明異議，復經本院於審理時逐一提示予被告表示意見，本院審酌該等證據之取得過程並無瑕疵，與本案待證事實間復具有相當之關聯性，以之為本案證據並無不當，依刑事訴訟法第159條之5規定自得採為本件認定事實之基礎。二、另其餘本判決所採之非供述證據亦均經法定程序取得，無不得為證據之情形，自均有證據能力。貳、實體部分：一、訊據被告陳榮煉固坦承因欲向案外人林采晴借款，在未經證人李美娟同意就以證人李美娟的名義簽立讓渡書，並將讓渡書交給案外人林采晴，讓渡書是案外人林采晴要求的，我當時是認為自己如果有還錢後這個讓渡書就無效，而且是證人李美娟一直說要叫我將車處理，所以我就簽了該讓渡書，我認為沒有造成損害等語（見本院卷第31頁、第32頁）。經查：(一)被告陳榮煉確未徵得證人李美娟同意即偽造「李美娟」之署名1枚填載於「讓渡證書」之「立讓渡書人」欄位，並將讓渡書作為向案外人林采晴借款之擔保一情，業據被告坦承不諱，核與證人即告訴人李美娟於偵查中以書面及本院證稱相符（見他字卷第58頁，本院卷第46頁），此外復有讓渡書1紙在卷可稽（見他字卷第101頁、第102頁），此部分事實應可認定。(二)又刑法上偽造文書罪之成立，以足生損害於公眾或他人為特別要

### 使用全部test facts在train facts中尋找最相近的五篇facts並計算預測刑期

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT_loaded.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / 5

    predicted_sentences_all.append(predicted_sentence)



In [ ]:
print(predicted_sentences_all)
print(test_sentences)

[111.0, 102.0, 144.0, 100.0, 162.0, 308.0, 180.0, 265.0, 156.0, 211.0, 186.0, 193.0, 132.0, 126.0, 214.0, 138.0, 289.0, 150.0, 203.0, 185.0, 205.0, 138.0, 314.0, 149.0, 100.0, 107.0, 180.4, 136.0, 138.0, 97.6, 126.0, 121.0, 214.6, 211.0, 156.0, 139.6, 113.0, 240.0, 138.0, 138.0, 100.0, 231.0, 144.0, 241.0, 162.0, 235.0, 332.0, 114.0, 88.0, 186.0, 313.2, 108.0, 173.0, 211.0, 93.0, 151.0, 365.2, 126.0, 174.4, 176.0, 99.0, 148.0, 105.0, 108.0, 136.0, 170.0, 290.0, 150.0, 202.0, 156.0, 116.0, 141.0, 156.0, 175.0, 84.0, 283.4, 150.0, 170.0, 126.0, 153.0, 166.0, 95.4, 156.0, 301.0, 227.0, 165.0, 102.0, 165.0, 187.0, 162.0, 175.0, 132.0, 101.0, 356.0, 187.0, 402.2, 174.0, 120.0, 146.0, 125.0]
[150, 242, 60, 302, 365, 150, 90, 120, 120, 180, 120, 90, 120, 90, 90, 90, 180, 270, 425, 50, 150, 90, 365, 195, 150, 50, 345, 150, 150, 90, 425, 135, 120, 240, 20, 240, 60, 120, 60, 150, 90, 240, 365, 210, 150, 120, 32, 150, 505, 90, 90, 120, 120, 120, 165, 180, 302, 90, 120, 730, 240, 40, 180, 90, 90, 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 21656.706400000003
Linear Regression - RMSE: 147.16217720596555
Linear Regression - MAE: 98.25200000000001


## colbert

In [ ]:
output_1 = model.encode(sword_1, return_dense=True, return_sparse=True, return_colbert_vecs=True)
output_2 = model.encode(sword_2, return_dense=True, return_sparse=True, return_colbert_vecs=True)

print(model.colbert_score(output_1['colbert_vecs'][0], output_2['colbert_vecs'][0]))
print(model.colbert_score(output_1['colbert_vecs'][0], output_2['colbert_vecs'][1]))
print(model.colbert_score(output_1['colbert_vecs'][1], output_2['colbert_vecs'][0]))
print(model.colbert_score(output_1['colbert_vecs'][1], output_2['colbert_vecs'][1]))

tensor(0.8653)
tensor(0.5597)
tensor(0.5299)
tensor(0.7482)


In [ ]:
print(output_1['colbert_vecs'][0].shape)
print(output_1['colbert_vecs'][1].shape)

(3, 1024)
(3, 1024)


In [ ]:
print(type(test_embeddings_colbert))
print(type(test_embeddings_colbert['colbert_vecs']))
print(type(test_embeddings_colbert['colbert_vecs'][0]))

<class 'dict'>
<class 'list'>
<class 'numpy.ndarray'>


In [ ]:
print(type(loaded_test_embeddings))
print(type(loaded_test_embeddings['colbert_vecs']))
print(type(loaded_test_embeddings['colbert_vecs'][0]))

<class 'dict'>
<class 'list'>
<class 'numpy.ndarray'>


In [ ]:
max_length_train = max(len(vec) for vec in loaded_train_embeddings['colbert_vecs'])
max_length_test = max(len(vec) for vec in loaded_test_embeddings['colbert_vecs'])
max_length = max(max_length_train, max_length_test)
max_length

1154

In [ ]:
def colbert_metric(vector1, vector2):
    # 将一维向量重塑为二维数组
    vector1_reshaped = vector1.reshape(max_length, 1024)
    vector2_reshaped = vector2.reshape(max_length, 1024)

    # 计算相似度
    colbert_similarity = model.colbert_score(vector1_reshaped, vector2_reshaped)
    # 计算距离（1 - 相似度）
    distance = 1 - colbert_similarity
    # 返回距离值
    return distance  # 这里使用平均值作为最终的距离值，您可以根据需要选择其他方法

In [ ]:
k = 5
knn_model_colbert = NearestNeighbors(n_neighbors=k, metric=colbert_metric)

In [ ]:
print(loaded_train_embeddings['colbert_vecs'][0].shape)
print(loaded_train_embeddings['colbert_vecs'][1].shape)
print(loaded_train_embeddings['colbert_vecs'][2].shape)

(194, 1024)
(315, 1024)
(441, 1024)


In [ ]:
padded_train_embeddings_colbert = []
for embedding in loaded_train_embeddings['colbert_vecs']:
  zero_embedding = np.zeros((max_length, 1024))
  for i, vec in enumerate(embedding):
    zero_embedding[i] = vec
  padded_train_embeddings_colbert.append(zero_embedding)

In [ ]:
print(padded_train_embeddings_colbert[0].shape)
print(padded_train_embeddings_colbert[1].shape)
print(padded_train_embeddings_colbert[2].shape)

(1154, 1024)
(1154, 1024)
(1154, 1024)


In [ ]:
reshape_padded_train_embeddings_colbert = np.array(padded_train_embeddings_colbert).reshape(500,-1)

In [ ]:
reshape_padded_train_embeddings_colbert.shape

(500, 1181696)

In [ ]:
knn_model_colbert.fit(reshape_padded_train_embeddings_colbert)

NearestNeighbors(metric=<function colbert_metric at 0x7da01831ab90>)

In [ ]:
# 將模型保存到文件
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/knn_model_colbert.pkl', 'wb') as f:
    pickle.dump(knn_model_colbert, f)

# 從文件加載模型
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/knn_model_colbert.pkl', 'rb') as f:
    knn_model_colbert_loaded = pickle.load(f)

PicklingError: Can't pickle <function colbert_metric at 0x7fb3c46c3400>: it's not the same object as __main__.colbert_metric

In [ ]:
padded_test_embeddings_colbert = []
for embedding in loaded_test_embeddings['colbert_vecs']:
  zero_embedding = np.zeros((max_length, 1024))
  for i, vec in enumerate(embedding):
    zero_embedding[i] = vec
  padded_test_embeddings_colbert.append(zero_embedding)

In [ ]:
print(padded_test_embeddings_colbert[0].shape)
print(padded_test_embeddings_colbert[1].shape)

(1154, 1024)
(1154, 1024)


In [ ]:
reshape_padded_test_embeddings_colbert = np.array(padded_test_embeddings_colbert).reshape(100,-1)

In [ ]:
print(reshape_padded_test_embeddings_colbert.shape)
print(reshape_padded_test_embeddings_colbert[0].shape)
print(reshape_padded_test_embeddings_colbert[0].reshape(-1, 1).shape)

(100, 1181696)
(1181696,)
(1181696, 1)


In [ ]:
distances, indices = knn_model_colbert.kneighbors(reshape_padded_test_embeddings_colbert)

In [ ]:
distances

array([[0.8467546 , 0.85628895, 0.85792951, 0.85798034, 0.85812114],
       [0.46786588, 0.46904526, 0.47064338, 0.4724483 , 0.47270239],
       [0.68826188, 0.69833034, 0.71697342, 0.71933531, 0.7221358 ],
       [0.9022876 , 0.90252518, 0.90296736, 0.90693968, 0.91095569],
       [0.89945079, 0.90220673, 0.90308042, 0.90334198, 0.90344348],
       [0.95062302, 0.95340193, 0.95599509, 0.95619578, 0.95635824],
       [0.73356978, 0.73466964, 0.73468267, 0.73665344, 0.7376735 ],
       [0.71970116, 0.72039855, 0.72337162, 0.72508263, 0.72585188],
       [0.93316376, 0.93319655, 0.93438942, 0.9345055 , 0.93488585],
       [0.92910114, 0.92966921, 0.92969095, 0.9298863 , 0.93010254],
       [0.91468292, 0.9166145 , 0.91728189, 0.91731523, 0.91746835],
       [0.87141812, 0.87774514, 0.88135079, 0.88160658, 0.88172145],
       [0.62877719, 0.63251745, 0.63353899, 0.63757815, 0.63969064],
       [0.88434062, 0.88694274, 0.89574548, 0.89687368, 0.89701152],
       [0.88715477, 0.88753215, 0.

In [ ]:
indices

array([[4, 5, 8],
       [2, 3, 4]])

In [ ]:
predicted_sentences_all = []
for nearest_articles_indices in indices:
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    predicted_sentences = 0
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / 5

    predicted_sentences_all.append(predicted_sentence)



In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 20205.824399999998
Linear Regression - RMSE: 142.14719272641298
Linear Regression - MAE: 93.43799999999999


# 平均

In [ ]:
avg_sentence = sum(train_sentences)/len(train_sentences)

In [ ]:
avg_sentence

187.724

In [ ]:
avg_sentences = [avg_sentence]*len(test_sentences)

In [ ]:
mse_linear = mean_squared_error(test_sentences, avg_sentences)
rmse_linear = mean_squared_error(test_sentences, avg_sentences, squared=False)
mae_linear = mean_absolute_error(test_sentences, avg_sentences)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16325.817136000003
Linear Regression - RMSE: 127.77252105206347
Linear Regression - MAE: 89.10199999999999
